<a href="https://colab.research.google.com/github/shalinisaini/CS60092_9A_18CS60R47/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>